# AirREGI ヘルプデスク入電予測 - EDA & 特徴量エンジニアリング

## 概要
このノートブックでは、時系列データの探索的データ分析と特徴量エンジニアリングを行います。

### 設計方針
- **モジュール化**: 各特徴量グループをクラスで管理
- **テスト容易性**: 各特徴量を独立してテスト可能
- **データリーケージ防止**: 時系列データで未来の情報を使わない
- **可読性**: コードの意図を明確に

In [27]:
# ライブラリのインポート
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import timedelta
from typing import List, Optional, Dict, Any
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定（文字化け対策）
plt.rcParams['font.sans-serif'] = ['MS Gothic', 'Yu Gothic', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 表示設定
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

print("Setup complete!")

Setup complete!


## 1. データ読み込み

In [28]:
class DataLoader:
    """データ読み込みと前処理を行うクラス"""
    
    def __init__(self, input_dir: str = '../input'):
        self.input_dir = input_dir
        
    def load_all(self) -> Dict[str, pd.DataFrame]:
        """全データを読み込み、日付型に変換"""
        print("=" * 80)
        print("データ読み込み開始")
        print("=" * 80)
        
        # データ読み込み
        calender = pd.read_csv(f'{self.input_dir}/calender_data.csv')
        cm_data = pd.read_csv(f'{self.input_dir}/cm_data.csv')
        gt_service = pd.read_csv(f'{self.input_dir}/gt_service_name.csv')
        acc_get = pd.read_csv(f'{self.input_dir}/regi_acc_get_data_transform.csv')
        call_data = pd.read_csv(f'{self.input_dir}/regi_call_data_transform.csv')
        
        # 日付カラムをdatetime型に変換
        calender['cdr_date'] = pd.to_datetime(calender['cdr_date'])
        cm_data['cdr_date'] = pd.to_datetime(cm_data['cdr_date'])
        acc_get['cdr_date'] = pd.to_datetime(acc_get['cdr_date'])
        call_data['cdr_date'] = pd.to_datetime(call_data['cdr_date'])
        gt_service['week'] = pd.to_datetime(gt_service['week'])
        
        # データサイズを表示
        datasets = {
            'calender': calender,
            'cm_data': cm_data,
            'gt_service': gt_service,
            'acc_get': acc_get,
            'call_data': call_data
        }
        
        print("\n読み込み完了:")
        for name, df in datasets.items():
            print(f"  {name:15s}: {df.shape}")
        
        return datasets
    
    def merge_all(self, datasets: Dict[str, pd.DataFrame]) -> pd.DataFrame:
        """全データを統合"""
        print("\n" + "=" * 80)
        print("データ統合開始")
        print("=" * 80)
        
        # メインデータ（入電数）を基準
        df = datasets['call_data'].copy()
        print(f"\nベースデータ: {df.shape}")
        
        # カレンダー情報をマージ
        df = df.merge(datasets['calender'], on='cdr_date', how='left')
        print(f"カレンダー統合後: {df.shape}")
        
        # CM情報をマージ
        df = df.merge(datasets['cm_data'], on='cdr_date', how='left')
        print(f"CM統合後: {df.shape}")
        
        # アカウント取得数をマージ
        df = df.merge(datasets['acc_get'], on='cdr_date', how='left')
        print(f"アカウント取得数統合後: {df.shape}")
        
        # Google Trendsを週次→日次に展開
        gt_daily = self._expand_weekly_to_daily(datasets['gt_service'])
        df = df.merge(gt_daily, on='cdr_date', how='left')
        print(f"Google Trends統合後: {df.shape}")
        
        # 日付でソート（時系列処理のため必須）
        df = df.sort_values('cdr_date').reset_index(drop=True)
        print("\n日付でソート完了（時系列処理のため）")
        
        # 欠損値確認
        print("\n欠損値の数（上位10）:")
        missing = df.isnull().sum().sort_values(ascending=False).head(10)
        for col, count in missing.items():
            if count > 0:
                print(f"  {col:30s}: {count:4d} ({count/len(df)*100:.1f}%)")
        
        return df
    
    @staticmethod
    def _expand_weekly_to_daily(gt_service: pd.DataFrame) -> pd.DataFrame:
        """週次データを日次に展開"""
        print("\nGoogle Trendsを週次→日次に展開中...")
        daily_records = []
        
        for _, row in gt_service.iterrows():
            week_start = row['week']
            for i in range(7):
                date = week_start + timedelta(days=i)
                daily_records.append({
                    'cdr_date': date,
                    'search_cnt': row['search_cnt']
                })
        
        return pd.DataFrame(daily_records)


# データ読み込み実行
loader = DataLoader()
datasets = loader.load_all()
df_raw = loader.merge_all(datasets)

print("\n" + "=" * 80)
print(f"統合データ: {df_raw.shape}")
print(f"期間: {df_raw['cdr_date'].min()} ~ {df_raw['cdr_date'].max()}")
print("=" * 80)


データ読み込み開始

読み込み完了:
  calender       : (670, 10)
  cm_data        : (762, 2)
  gt_service     : (109, 2)
  acc_get        : (701, 2)
  call_data      : (670, 2)

データ統合開始

ベースデータ: (670, 2)
カレンダー統合後: (670, 11)
CM統合後: (670, 12)
アカウント取得数統合後: (670, 13)

Google Trendsを週次→日次に展開中...
Google Trends統合後: (670, 14)

日付でソート完了（時系列処理のため）

欠損値の数（上位10）:
  holiday_name                  :  632 (94.3%)

統合データ: (670, 14)
期間: 2018-06-01 00:00:00 ~ 2020-03-31 00:00:00


## 2. 特徴量エンジニアリング

### 設計パターン
各特徴量グループを独立したクラスとして実装し、テストと保守を容易にします。

In [29]:
from abc import ABC, abstractmethod

class BaseFeatureEngineer(ABC):
    """特徴量エンジニアリングの基底クラス"""
    
    def __init__(self, name: str):
        self.name = name
        self.created_features: List[str] = []
    
    @abstractmethod
    def create_features(self, df: pd.DataFrame) -> pd.DataFrame:
        """特徴量を作成（サブクラスで実装）"""
        pass
    
    def get_feature_names(self) -> List[str]:
        """作成された特徴量名のリストを取得"""
        return self.created_features
    
    def describe(self, df: pd.DataFrame) -> pd.DataFrame:
        """特徴量の統計情報を取得"""
        if not self.created_features:
            print(f"{self.name}: 特徴量が未作成です")
            return pd.DataFrame()
        return df[self.created_features].describe()


class TimeBasedFeatures(BaseFeatureEngineer):
    """日付から派生する基本的な時系列特徴量
    
    これらは未来の情報を使わないため、データリーケージの心配がありません。
    """
    
    def __init__(self):
        super().__init__("時系列基本特徴量")
    
    def create_features(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        
        # 年月日の特徴量
        df['year'] = df['cdr_date'].dt.year
        df['month'] = df['cdr_date'].dt.month
        df['day_of_month'] = df['cdr_date'].dt.day
        df['quarter'] = df['cdr_date'].dt.quarter
        df['day_of_year'] = df['cdr_date'].dt.dayofyear
        df['week_of_year'] = df['cdr_date'].dt.isocalendar().week
        
        # 経過日数
        df['days_from_start'] = (df['cdr_date'] - df['cdr_date'].min()).dt.days
        
        # 月初・月末フラグ
        df['is_month_start'] = (df['day_of_month'] <= 5).astype(int)
        df['is_month_end'] = (df['day_of_month'] >= 25).astype(int)
        
        # 週初・週末（既存のdowを利用）
        if 'dow' in df.columns:
            df['is_week_start'] = (df['dow'] == 1).astype(int)  # 月曜
            df['is_week_end'] = (df['dow'] == 5).astype(int)    # 金曜
        
        self.created_features = [
            'year', 'month', 'day_of_month', 'quarter', 'day_of_year',
            'week_of_year', 'days_from_start', 'is_month_start', 'is_month_end',
            'is_week_start', 'is_week_end'
        ]
        
        print(f"{self.name}: {len(self.created_features)}個の特徴量を作成")
        return df


class LagFeatures(BaseFeatureEngineer):
    """ラグ特徴量（過去のデータ）
    
    重要:
    - shift()を使って未来の情報が混入しないようにする
    - データは日付順にソート済みであることが前提
    """
    
    def __init__(self, target_col: str = 'call_num', lags: List[int] = [1, 2, 3, 5, 7, 14, 30]):
        super().__init__("ラグ特徴量")
        self.target_col = target_col
        self.lags = lags
    
    def create_features(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        
        if self.target_col not in df.columns:
            print(f"警告: {self.target_col}が見つかりません")
            return df
        
        for lag in self.lags:
            col_name = f'lag_{lag}'
            df[col_name] = df[self.target_col].shift(lag)
            self.created_features.append(col_name)
        
        print(f"{self.name}: {len(self.created_features)}個の特徴量を作成")
        print(f"  対象変数: {self.target_col}")
        print(f"  ラグ: {self.lags}")
        print(f"  注意: 最初の{max(self.lags)}日間はNaNになります")
        
        return df


class RollingFeatures(BaseFeatureEngineer):
    """移動統計量特徴量（移動平均、移動標準偏差など）
    
    重要:
    - rolling()の前にshift(1)を適用してデータリーケージを防止
    - 当日のデータが含まれないようにする
    """
    
    def __init__(self, target_col: str = 'call_num', windows: List[int] = [3, 7, 14, 30]):
        super().__init__("移動統計量特徴量")
        self.target_col = target_col
        self.windows = windows
    
    def create_features(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        
        if self.target_col not in df.columns:
            print(f"警告: {self.target_col}が見つかりません")
            return df
        
        for window in self.windows:
            # 移動平均（当日を含まない）
            ma_col = f'ma_{window}'
            df[ma_col] = df[self.target_col].shift(1).rolling(
                window=window, min_periods=1
            ).mean()
            self.created_features.append(ma_col)
            
            # 移動標準偏差（変動性を捉える）
            std_col = f'ma_std_{window}'
            df[std_col] = df[self.target_col].shift(1).rolling(
                window=window, min_periods=1
            ).std()
            self.created_features.append(std_col)
            
            # 移動最大値
            max_col = f'ma_max_{window}'
            df[max_col] = df[self.target_col].shift(1).rolling(
                window=window, min_periods=1
            ).max()
            self.created_features.append(max_col)
            
            # 移動最小値
            min_col = f'ma_min_{window}'
            df[min_col] = df[self.target_col].shift(1).rolling(
                window=window, min_periods=1
            ).min()
            self.created_features.append(min_col)
        
        print(f"{self.name}: {len(self.created_features)}個の特徴量を作成")
        print(f"  対象変数: {self.target_col}")
        print(f"  ウィンドウ: {self.windows}")
        print(f"  統計量: 平均, 標準偏差, 最大値, 最小値")
        
        return df


class DomainFeatures(BaseFeatureEngineer):
    """ドメイン知識に基づく特徴量
    
    - CM効果の累積
    - Google Trendsの平滑化
    - アカウント取得数の傾向
    - 曜日ごとの過去平均
    """
    
    def __init__(self):
        super().__init__("ドメイン特徴量")
    
    def create_features(self, df: pd.DataFrame) -> pd.DataFrame:
        df = df.copy()
        
        # CM効果の累積（過去7日間のCM実施回数）
        if 'cm_flg' in df.columns:
            df['cm_7d_sum'] = df['cm_flg'].shift(1).rolling(window=7, min_periods=1).sum()
            df['cm_14d_sum'] = df['cm_flg'].shift(1).rolling(window=14, min_periods=1).sum()
            self.created_features.extend(['cm_7d_sum', 'cm_14d_sum'])
        
        # Google Trendsの移動平均（ノイズ除去）
        if 'search_cnt' in df.columns:
            df['gt_ma_7'] = df['search_cnt'].shift(1).rolling(window=7, min_periods=1).mean()
            df['gt_ma_14'] = df['search_cnt'].shift(1).rolling(window=14, min_periods=1).mean()
            self.created_features.extend(['gt_ma_7', 'gt_ma_14'])
        
        # アカウント取得数の移動平均
        if 'acc_get_cnt' in df.columns:
            df['acc_ma_7'] = df['acc_get_cnt'].shift(1).rolling(window=7, min_periods=1).mean()
            df['acc_ma_14'] = df['acc_get_cnt'].shift(1).rolling(window=14, min_periods=1).mean()
            self.created_features.extend(['acc_ma_7', 'acc_ma_14'])
        
        # 曜日ごとの過去平均（同じ曜日のパターンを捉える）
        if 'dow' in df.columns and 'call_num' in df.columns:
            df['dow_avg'] = np.nan
            for dow in df['dow'].unique():
                mask = df['dow'] == dow
                df.loc[mask, 'dow_avg'] = df.loc[mask, 'call_num'].shift(1).expanding().mean()
            self.created_features.append('dow_avg')
        
        print(f"{self.name}: {len(self.created_features)}個の特徴量を作成")
        return df


print("特徴量エンジニアリングクラス定義完了")
df.columns

特徴量エンジニアリングクラス定義完了


Index(['cdr_date', 'call_num'], dtype='object')

In [30]:
# セル4を実行済みなら、これで確認できる
print(type(datasets))  # <class 'dict'>
print(datasets.keys())  # dict_keys(['calender', 'cm_data', 'gt_service', 'acc_get', 'call_data'])

# calenderデータにアクセス
print(datasets['calender']['cdr_date'].dtype)  # datetime64[ns]
print(datasets['calender']['cdr_date'].head())

<class 'dict'>
dict_keys(['calender', 'cm_data', 'gt_service', 'acc_get', 'call_data'])
datetime64[ns]
0   2018-06-01
1   2018-06-02
2   2018-06-03
3   2018-06-04
4   2018-06-05
Name: cdr_date, dtype: datetime64[ns]


In [31]:
import os

# 保存先ディレクトリの作成
output_dir = '../output/datasets'
os.makedirs(output_dir, exist_ok=True)

# 各DataFrameをCSVとして保存
for name, df in datasets.items():
    filepath = f'{output_dir}/{name}.csv'
df.to_csv(filepath, index=False, encoding='utf-8')
print(f"保存完了: {filepath} ({df.shape})")

保存完了: ../output/datasets/call_data.csv ((670, 2))


In [32]:
"""
データ構造を確認するスクリプト
EDAノートブックのセル4を実行した後に実行してください
"""

import pandas as pd
from datetime import timedelta

# ==========================================
# サンプルデータで説明
# ==========================================

print("=" * 80)
print("データ構造の説明（サンプルデータ）")
print("=" * 80)

# 1. 個別データ（結合前）
print("\n" + "=" * 80)
print("1. 結合前の個別データ（datasets）")
print("=" * 80)

call_data = pd.DataFrame({
    'cdr_date': pd.to_datetime(['2018-06-01', '2018-06-02', '2018-06-03']),
    'call_num': [183, 0, 96]
})
print("\n[call_data] - 入電数（メインデータ）")
print(call_data)
print(f"shape: {call_data.shape} （{call_data.shape[0]}行 × {call_data.shape[1]}列）")
print(f"columns: {call_data.columns.tolist()}")

calender = pd.DataFrame({
    'cdr_date': pd.to_datetime(['2018-06-01', '2018-06-02', '2018-06-03']),
    'dow': [5, 6, 7],
    'dow_name': ['Friday', 'Saturday', 'Sunday'],
    'holiday_flag': [0, 0, 0]
})
print("\n[calender] - カレンダー情報")
print(calender)
print(f"shape: {calender.shape} （{calender.shape[0]}行 × {calender.shape[1]}列）")
print(f"columns: {calender.columns.tolist()}")

cm_data = pd.DataFrame({
    'cdr_date': pd.to_datetime(['2018-06-01', '2018-06-02', '2018-06-03']),
    'cm_flg': [0, 1, 0]
})
print("\n[cm_data] - CM実施フラグ")
print(cm_data)
print(f"shape: {cm_data.shape} （{cm_data.shape[0]}行 × {cm_data.shape[1]}列）")
print(f"columns: {cm_data.columns.tolist()}")

# 2. 結合処理
print("\n" + "=" * 80)
print("2. 結合処理（merge）")
print("=" * 80)

df = call_data.copy()
print(f"\nステップ1: call_dataをコピー")
print(f"  shape: {df.shape}")
print(f"  columns: {df.columns.tolist()}")

df = df.merge(calender, on='cdr_date', how='left')
print(f"\nステップ2: calenderを結合")
print(f"  shape: {df.shape}")
print(f"  columns: {df.columns.tolist()}")
print("  ↑ call_dataのカラム + calenderのカラム（cdr_date以外）")

df = df.merge(cm_data, on='cdr_date', how='left')
print(f"\nステップ3: cm_dataを結合")
print(f"  shape: {df.shape}")
print(f"  columns: {df.columns.tolist()}")
print("  ↑ さらにcm_dataのカラム（cdr_date以外）を追加")

# 3. 結合後のデータ
print("\n" + "=" * 80)
print("3. 結合後のデータ（df_raw）")
print("=" * 80)

print("\n[df_raw] - 全データが横に結合された1つのテーブル")
print(df)
print(f"\nshape: {df.shape} （{df.shape[0]}行 × {df.shape[1]}列）")
print(f"columns: {df.columns.tolist()}")

# 4. データの取り出し方
print("\n" + "=" * 80)
print("4. データの取り出し方")
print("=" * 80)

print("\n■ 1つの列を取り出す")
print("df['call_num']")
print(df['call_num'])

print("\n■ 複数の列を取り出す")
print("df[['cdr_date', 'call_num', 'dow']]")
print(df[['cdr_date', 'call_num', 'dow']])

print("\n■ 1行目のデータ")
print("df.iloc[0]")
print(df.iloc[0])

print("\n■ 特定の値")
print("df.loc[0, 'call_num']  # 1行目のcall_num")
print(df.loc[0, 'call_num'])

# 5. df.columnsの説明
print("\n" + "=" * 80)
print("5. df.columnsの説明")
print("=" * 80)

print(f"\ndf.columns = {df.columns}")
print(f"型: {type(df.columns)}")
print(f"データ型: {df.columns.dtype}")

print("\n■ 列名をリストとして取得")
print(f"df.columns.tolist() = {df.columns.tolist()}")

print("\n■ 列数")
print(f"len(df.columns) = {len(df.columns)}")

print("\n■ 列名を1つずつ表示")
for i, col in enumerate(df.columns):
    print(f"  {i}: {col}")

# 6. 実際のテーブル構造を視覚化
print("\n" + "=" * 80)
print("6. テーブル構造の視覚化")
print("=" * 80)

print("\n結合前:")
print("""
call_data          calender              cm_data
┌────────────┐    ┌─────────────────┐    ┌───────────┐
│ cdr_date   │    │ cdr_date        │    │ cdr_date  │
│ call_num   │    │ dow             │    │ cm_flg    │
└────────────┘    │ dow_name        │    └───────────┘
   2列            │ holiday_flag    │       2列
                  └─────────────────┘
                        4列
""")

print("\n結合後（横に並べる）:")
print("""
df_raw
┌───────────────────────────────────────────────────┐
│ cdr_date  call_num  dow  dow_name  holiday_flag  cm_flg │
└───────────────────────────────────────────────────┘
                    6列（全て横に並ぶ）
""")

print("\n行の構造:")
for idx, row in df.iterrows():
    print(f"\n{idx}行目:")
    print(f"  cdr_date     : {row['cdr_date']}")
    print(f"  call_num     : {row['call_num']}")
    print(f"  dow          : {row['dow']}")
    print(f"  dow_name     : {row['dow_name']}")
    print(f"  holiday_flag : {row['holiday_flag']}")
    print(f"  cm_flg       : {row['cm_flg']}")

print("\n" + "=" * 80)
print("説明完了")
print("=" * 80)


データ構造の説明（サンプルデータ）

1. 結合前の個別データ（datasets）

[call_data] - 入電数（メインデータ）
    cdr_date  call_num
0 2018-06-01       183
1 2018-06-02         0
2 2018-06-03        96
shape: (3, 2) （3行 × 2列）
columns: ['cdr_date', 'call_num']

[calender] - カレンダー情報
    cdr_date  dow  dow_name  holiday_flag
0 2018-06-01    5    Friday             0
1 2018-06-02    6  Saturday             0
2 2018-06-03    7    Sunday             0
shape: (3, 4) （3行 × 4列）
columns: ['cdr_date', 'dow', 'dow_name', 'holiday_flag']

[cm_data] - CM実施フラグ
    cdr_date  cm_flg
0 2018-06-01       0
1 2018-06-02       1
2 2018-06-03       0
shape: (3, 2) （3行 × 2列）
columns: ['cdr_date', 'cm_flg']

2. 結合処理（merge）

ステップ1: call_dataをコピー
  shape: (3, 2)
  columns: ['cdr_date', 'call_num']

ステップ2: calenderを結合
  shape: (3, 5)
  columns: ['cdr_date', 'call_num', 'dow', 'dow_name', 'holiday_flag']
  ↑ call_dataのカラム + calenderのカラム（cdr_date以外）

ステップ3: cm_dataを結合
  shape: (3, 6)
  columns: ['cdr_date', 'call_num', 'dow', 'dow_name', 'holiday_flag', '

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# データ読み込み
cm_data = pd.read_csv('../input/cm_data.csv')
acc_get = pd.read_csv('../input/regi_acc_get_data_transform.csv')
call_data = pd.read_csv('../input/regi_call_data_transform.csv')

# 日付型に変換
cm_data['cdr_date'] = pd.to_datetime(cm_data['cdr_date'])
acc_get['cdr_date'] = pd.to_datetime(acc_get['cdr_date'])
call_data['cdr_date'] = pd.to_datetime(call_data['cdr_date'])

# マージ
df = call_data.merge(cm_data, on='cdr_date', how='left')
df = df.merge(acc_get, on='cdr_date', how='left')

# ========================================
# Plot 1: 時系列で比較（2軸）
# ========================================
fig, ax1 = plt.subplots(figsize=(14, 5))

# CM放送日を縦線で表示
cm_dates = df[df['cm_flg'] == 1]['cdr_date']
for d in cm_dates:
    ax1.axvline(x=d, color='red', alpha=0.3, linewidth=1)

# アカウント取得数
ax1.plot(df['cdr_date'], df['acc_get_cnt'], color='blue', alpha=0.7, label='acc_get_cnt')
ax1.set_xlabel('Date')
ax1.set_ylabel('acc_get_cnt', color='blue')
ax1.tick_params(axis='y', labelcolor='blue')

# 凡例用のダミー
ax1.axvline(x=df['cdr_date'].min(), color='red', alpha=0.5, linewidth=2, label='CM broadcast')

ax1.legend(loc='upper left')
ax1.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
plt.xticks(rotation=45)
plt.title('CM Broadcast vs Account Acquisition (Time Series)')
plt.tight_layout()
plt.show()

# ========================================
# Plot 2: CM前後のacc_get_cnt比較（箱ひげ図）
# ========================================
fig, ax = plt.subplots(figsize=(8, 5))

# CM放送日とそれ以外で分類
df['cm_label'] = df['cm_flg'].map({0: 'No CM', 1: 'CM Day'})
df.boxplot(column='acc_get_cnt', by='cm_label', ax=ax)
plt.suptitle('')
plt.title('acc_get_cnt: CM Day vs No CM Day')
plt.xlabel('')
plt.ylabel('acc_get_cnt')
plt.tight_layout()
plt.show()

# ========================================
# Plot 3: CM放送後のacc_get_cnt推移（イベントスタディ風）
# ========================================
# CM放送日を基準に前後N日のacc_get_cntを集計
window = 14  # CM前後14日

cm_indices = df[df['cm_flg'] == 1].index.tolist()
effects = []

for idx in cm_indices:
    for offset in range(-window, window + 1):
        target_idx = idx + offset
        if 0 <= target_idx < len(df):
            effects.append({
                'offset': offset,
                'acc_get_cnt': df.loc[target_idx, 'acc_get_cnt']
            })

effects_df = pd.DataFrame(effects)
avg_effect = effects_df.groupby('offset')['acc_get_cnt'].mean()

fig, ax = plt.subplots(figsize=(10, 5))
ax.bar(avg_effect.index, avg_effect.values, color=['red' if x == 0 else 'steelblue' for x in avg_effect.index])
ax.axvline(x=0, color='red', linestyle='--', linewidth=2, label='CM Day')
ax.set_xlabel('Days from CM Broadcast')
ax.set_ylabel('avg acc_get_cnt')
ax.set_title(f'Average acc_get_cnt around CM Broadcast (±{window} days)')
ax.legend()
plt.tight_layout()
plt.show()

# ========================================
# Plot 4: 相関散布図
# ========================================
# CM放送の累積効果（過去7日間のCM回数）
df['cm_7d'] = df['cm_flg'].rolling(window=7, min_periods=1).sum()

fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# 当日CM vs acc_get_cnt
axes[0].scatter(df['cm_flg'], df['acc_get_cnt'], alpha=0.5)
axes[0].set_xlabel('cm_flg (0 or 1)')
axes[0].set_ylabel('acc_get_cnt')
axes[0].set_title('CM Flag vs acc_get_cnt')

# 過去7日CM回数 vs acc_get_cnt
axes[1].scatter(df['cm_7d'], df['acc_get_cnt'], alpha=0.5)
axes[1].set_xlabel('CM count in past 7 days')
axes[1].set_ylabel('acc_get_cnt')
axes[1].set_title('CM (7d cumulative) vs acc_get_cnt')

plt.tight_layout()
plt.show()

# ========================================
# 統計サマリ
# ========================================
print("\n=== CM Day vs No CM Day: acc_get_cnt Stats ===")
print(df.groupby('cm_label')['acc_get_cnt'].describe())

print("\n=== Correlation ===")
print(f"cm_flg vs acc_get_cnt: {df['cm_flg'].corr(df['acc_get_cnt']):.4f}")
print(f"cm_7d vs acc_get_cnt:  {df['cm_7d'].corr(df['acc_get_cnt']):.4f}")


FileNotFoundError: [Errno 2] No such file or directory: '../input/cm_data.csv'